In [302]:
import pandas as pd
import re
import numpy as np
import math

In [303]:
data = pd.read_csv("../CSC_Dapp_SC_scraping/dapps_with_SC_parsed.csv")

In [304]:
# less than 60 cases -> other
for col in ['property', 'governance', 'wallet', 'security', 'identity', 'storage', 'health', 'energy']:
    data.loc[data['category']==col, 'category'] = 'other'

In [305]:
def prep(row):
    if not pd.isnull(row):
        row = re.sub('\n', ' newline ', row)
        row = re.sub('\t', ' tab ', row)
        row = re.sub('\r', ' cr ', row)
        row = re.sub('/', '', row)
        row = re.sub('\*', '', row)
        row = re.sub(';', '', row)
        row = re.sub(' +', ' ', row)
    return row

data.SC_full = data.SC_full.apply(prep)
data.SC_no_comments = data.SC_no_comments.apply(prep)
data.comments = data.comments.apply(prep)

In [306]:
data['pragma'] = data['SC_no_comments'].apply(lambda x: re.findall(r'pragma solidity \S+', x))
data['pragma'] = data['pragma'].apply(lambda x: x[0].split()[2] if len(x)!=0 else np.nan)
data['pragma'] = data['pragma'].apply(lambda x: re.sub('\D', '', x) if isinstance(x, str) else np.nan)

In [307]:
data['len'] = data['SC_no_comments'].apply(lambda x: len(x.split()))
data['comments_len'] = data['comments'].apply(lambda x: len(x.split()) if isinstance(x, str) else np.nan)
data['fun_count'] = data['SC_no_comments'].apply(lambda x: len(re.findall(r'function', x)))
data['event_count'] = data['SC_no_comments'].apply(lambda x: len(re.findall(r'event', x)))
data['contract_count'] = data['SC_no_comments'].apply(lambda x: len(re.findall(r'contract', x)))
data['comments'].fillna('no comments', inplace=True)

In [308]:
data.name_from_SC.fillna(data.slug, inplace=True)
data.loc[249,'name_from_SC'] = '_200eth'

In [309]:
for index, row in data.iterrows():
    for col in ['slug', 'description', 'category', 'created', 'name', 'rank', 'teaser']:
        if isinstance(row[col], (float, int, str)):
            data.loc[index, col] = data.loc[data['name_from_SC']==row['name_from_SC'], col].iloc[0]
        else:
            continue

In [310]:
data = data[~data.category.isna()]

In [311]:
data.SC_full

0       pragma solidity ^0.4.13 newline newline newlin...
1        hevm: flattened sources of nixstore8xb41r4qd0...
2       pragma solidity ^0.4.8 newline newline newline...
3       pragma solidity ^0.4.24 newline newline File: ...
4        newline newline newline newline ______ _ ____...
                              ...                        
2200    pragma solidity ^0.4.21 newline newline newlin...
2201     Degens Protocol (C) degens.com newline newlin...
2202     File: contractstokeninterfacesIERC20Token.sol...
2203     newline newline Copyright 2017-2019, bZeroX, ...
2204    pragma solidity ^0.4.24 newline newline newlin...
Name: SC_full, Length: 2146, dtype: object

In [312]:
data.to_csv('dapps_with_features_v4.csv', index=False)